In [1]:
!pip install transformers[torch]
!pip install -U datasets
!pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.2
    Uninstalling fsspec-2023.12.2:
      Successfully uninstalled fsspec-2023.12.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.

In [2]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "1.0.0")

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [4]:
from transformers import pipeline

sample_text = dataset["train"][1]["article"][:1000]
summaries = {}

2024-01-30 14:46:29.759128: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 14:46:29.759259: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 14:46:29.916076: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [20]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

string = "The U.S. are a country. The U.N. is an organization."
sent_tokenize(string)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['The U.S. are a country.', 'The U.N. is an organization.']

In [21]:
def three_sentence_summary(sentence):
  return '\n'.join(sent_tokenize(sentence)[:3])

summaries["baseline"] = three_sentence_summary(sample_text)

In [5]:
import torch

device="cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [23]:
pipe_mt5 = pipeline("summarization", model="google/mt5-large", device=device)

In [26]:
pipe_out = pipe_mt5(sample_text)
summaries["mt5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

In [24]:
pipe_bart = pipeline("summarization", model="facebook/bart-large", device=device)

In [27]:
len(sent_tokenize(sample_text))

7

In [28]:
pipe_out = pipe_bart(sample_text)
summaries["mbart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

In [30]:
pipe_mt5_m = pipeline("summarization", model="csebuetnlp/mT5_multilingual_XLSum", device=device)

In [31]:
pipe_out = pipe_mt5_m(sample_text)
summaries["mt5-multilingual"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

In [33]:
pipe_pegasus = pipeline("summarization", model="google/pegasus-cnn_dailymail", device=device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
pipe_out = pipe_pegasus(sample_text)
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")

In [37]:
print("INPUT")
print(sample_text)
print()
print("GROUND TRUTH")
print(dataset["train"][0]["highlights"])
print()
for model_name in summaries:
 print(model_name.upper())
 print(summaries[model_name])
 print()


INPUT
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won't do what they're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to foll

In [39]:
!pip install sacrebleu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 886.4 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 1.8 MB/s eta 0:00:00 0:00:01


In [8]:
from datasets import list_metrics, load_metric
bleu_metric = load_metric("sacrebleu")
print(bleu_metric)

/tmp/ipykernel_34/2877241137.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("sacrebleu")
/opt/conda/lib/python3.10/site-packages/datasets/load.py:752: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


ImportError: To be able to use sacrebleu, you need to install the following dependency: sacrebleu.
Please install it using 'pip install sacrebleu' for instance.

In [42]:
import pandas as pd

bleu_metric.add(prediction="the the the the the the", reference=["the cat is on the mat"])
pd.DataFrame.from_dict(bleu_metric.compute(smooth_method='floor', smooth_value=0), orient="index", columns=["Value"])

,Value
score,0.0
counts,"[4, 0, 0, 0]"
totals,"[12, 10, 8, 6]"
precisions,"[33.333333333333336, 0.0, 0.0, 0.0]"
bp,1.0
sys_len,12
ref_len,12


In [6]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=733980756d32e7b13b08929ad2b39131dc2601f8a2ed7a37137e7a03be886591
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [9]:
rouge_metric = load_metric("rouge", cache_dir=None)
rouge_metric


/opt/conda/lib/python3.10/site-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

In [11]:
import pandas as pd

reference = dataset["train"][1]["highlights"]
records = []
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
for model_name in summaries:
 rouge_metric.add(prediction=summaries[model_name], reference=reference)
 score = rouge_metric.compute()
 rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
 records.append(rouge_dict)
pd.DataFrame.from_records(records, index=summaries.keys())

ValueError: Must pass non-zero number of levels/codes

In [12]:
def evaluate_summaries_baseline(dataset, metric, column_text="article", column_summary="highlights"):
 summaries = [three_sentence_summary(text) for text in dataset[column_text]]
 metric.add_batch(predictions=summaries,references=dataset[column_summary])
 score = metric.compute()
 return score

In [18]:
test_sampled = dataset["test"].shuffle(seed=42).select(range(100))
score = evaluate_summaries_baseline(test_sampled, rouge_metric)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame.from_dict(rouge_dict, orient="index", columns= ["baseline"]).T

NameError: name 'three_sentence_summary' is not defined

In [25]:
from tqdm import tqdm
def chunks(lst, n):
 """Yield successive n-sized chunks from lst."""
 for i in range(0, len(lst), n):
     yield lst[i : i + n]
def evaluate_summaries_pegasus(dataset, metric, model, tokenizer, batch_size=16, device=device, column_text="article", column_summary="highlights"):
 article_batches = list(chunks(dataset[column_text], batch_size))
 target_batches = list(chunks(dataset[column_summary], batch_size))
 for article_batch, target_batch in tqdm(zip(article_batches, target_batches),total=len(article_batches)):
     dct = tokenizer.batch_encode_plus(article_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
     summaries = model.generate(input_ids=dct["input_ids"].to(device), length_penalty=0.8, attention_mask=dct["attention_mask"].to(device), num_beams=8,max_length=128)
     dec = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True)for g in summaries]
     dec = [d.replace("<n>", " ") for d in dec]
     metric.add_batch(predictions=dec, references=target_batch)
     report_gpu()
 score = metric.compute()
 return score


In [15]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
model_name = "google/pegasus-cnn_dailymail"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = (PegasusForConditionalGeneration.from_pretrained(model_name).to(device))

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [26]:
import gc 
def report_gpu(): 
    print(torch.cuda.list_gpu_processes()) 
    gc.collect() 
    torch.cuda.empty_cache()

In [28]:
report_gpu()

GPU:0
process       3146 uses    15082.000 MB GPU memory


In [29]:
score = evaluate_summaries_pegasus(dataset=test_sampled, model=model, tokenizer=tokenizer, metric=rouge_metric,batch_size=1)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame.from_dict(rouge_dict, orient="index", columns=["pegasus"]).T

  1%|          | 1/100 [00:02<04:41,  2.84s/it]

GPU:0
process       3146 uses     3776.000 MB GPU memory


  2%|▏         | 2/100 [00:05<04:37,  2.83s/it]

GPU:0
process       3146 uses     3838.000 MB GPU memory


  3%|▎         | 3/100 [00:09<04:57,  3.07s/it]

...

 99%|█████████▉| 99/100 [04:22<00:02,  2.41s/it]

GPU:0
process       3146 uses     3716.000 MB GPU memory


100%|██████████| 100/100 [04:25<00:00,  2.66s/it]

GPU:0
process       3146 uses     3876.000 MB GPU memory


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.426056,0.202636,0.300436,0.300433
